In [9]:
import pandas as pd

import snowflake.connector

import numpy as np

%matplotlib inline

import matplotlib.pyplot as plt 

from pydataset import data

from datetime import datetime, timedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

import config as cfg

snowflake_user = cfg.snowflake_access['snowflake_user']
snowflake_pass = cfg.snowflake_access['snowflake_pass']
snowflake_acct = cfg.snowflake_access['snowflake_acct']
snowflake_wh = 'BIRDFACTS_PROD_WAREHOUSE'
snowflake_db = 'BIRDFACTSDEV'
snowflake_schema = 'BIRDFACTS_ANALYTICS'

con = snowflake.connector.connect(
    user = snowflake_user,
    password = snowflake_pass,
    account = snowflake_acct,
    warehouse = snowflake_wh,
    database = snowflake_db,
    schema = snowflake_schema
)

cs = con.cursor()

list of emails from simon (through Adrianna)
Unsubscribed? column should be equal to False for the list of emails we need

In [123]:
simon_list = pd.read_csv("/Users/vamsi/Downloads/export-Records W Only Email-2020-05-22.csv")

In [140]:
simon_list_subscribed = simon_list[simon_list['Unsubscribed?'] == False]

In [141]:
print(simon_list_subscribed.shape)
simon_list_subscribed.head()

(177730, 2)


,email,Unsubscribed?
1,adammico@yahoo.com,False
2,cheriehornsby@mac.com,False
4,dhyoumans@gmail.com,False
5,jkockos07@gmail.com,False
8,dustinsiegel@gmail.com,False


customers in US that made a retail purchase at one of our stores, do not have a shipping address, and do not exist in our experian database

In [7]:
emails_list_of_customers_without_shipping_address_query = """
with emails_list_of_customers_without_shipping_address as
  (select distinct email from birdfactsdev.birdfacts_analytics.dim_customer
    where id in
  (
    select distinct customer_id from birdfactsdev.birdfacts_analytics.fact_sales
    where profit_center_country = 'United States'
    and sales_channel = 'Retail Brick & Mortar'
    and shipping_address_id in
    (
      select distinct shipping_address_id
      from
      (
      select profit_center_name, shipping_address_id, count(distinct order_name) as num_orders
      from
      birdfactsdev.birdfacts_analytics.fact_sales
      where profit_center_country = 'United States'
      and sales_channel = 'Retail Brick & Mortar'
      group by 1, 2
      order by 3 desc
      )
      where num_orders > 10
      and shipping_address_id is not NULL
    )
  )
  and email is not NULL
)
select 
row_number() over(partition by 1 order by 1)  as SEQUENCE_NUMBER,
email,
NULL as FIRST_NAME,
NULL as LAST_NAME,
NULL as BILLING_ADDRESS_LINE_1,
NULL as BILLING_ADDRESS_LINE_2,
NULL as BILLING_CITY,
NULL as BILLING_ZIP_CODE,
NULL as BILLING_STATE,
NULL as BILLING_COUNTRY,
NULL as SHIPPING_ADDRESS_LINE_1,
NULL as SHIPPING_ADDRESS_LINE_2,
NULL as SHIPPING_CITY,
NULL as SHIPPING_ZIP_CODE,
NULL as SHIPPING_STATE,
NULL as SHIPPING_COUNTRY
from emails_list_of_customers_without_shipping_address where email not in (select distinct email from birdfactsdev.birdfacts_analytics.experian_data)
"""

In [10]:
emails_list_of_customers_without_shipping_address = pd.DataFrame(cs.execute(emails_list_of_customers_without_shipping_address_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [20]:
print(emails_list_of_customers_without_shipping_address.shape)
emails_list_of_customers_without_shipping_address.head()

(207978, 16)


,SEQUENCE_NUMBER,EMAIL,FIRST_NAME,LAST_NAME,BILLING_ADDRESS_LINE_1,BILLING_ADDRESS_LINE_2,BILLING_CITY,BILLING_ZIP_CODE,BILLING_STATE,BILLING_COUNTRY,SHIPPING_ADDRESS_LINE_1,SHIPPING_ADDRESS_LINE_2,SHIPPING_CITY,SHIPPING_ZIP_CODE,SHIPPING_STATE,SHIPPING_COUNTRY
0,1,fuliwang2003@yahoo.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,eric.shepherd1104@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,3,rfharty@comcast.net,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,4,timothy.s.keyes5@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,5,johnsonnli1111@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None


customers in US that made a retail purchase at one of our stores, have a shipping address, and do not exist in our experian database

In [16]:
emails_list_of_customers_with_shipping_address_query = """
with emails_list_of_customers_with_shipping_address as
(
select 
b.email,
b.FIRST_NAME,
b.LAST_NAME,
c.ADDRESS_LINE_1 AS BILLING_ADDRESS_LINE_1,
c.ADDRESS_LINE_2 AS BILLING_ADDRESS_LINE_2,
c.CITY as BILLING_CITY,
c.POSTAL_CODE as BILLING_ZIP_CODE,
c.REGION as BILLING_STATE,
c.COUNTRY as BILLING_COUNTRY,
d.ADDRESS_LINE_1 AS SHIPPING_ADDRESS_LINE_1,
d.ADDRESS_LINE_2 AS SHIPPING_ADDRESS_LINE_2,
d.CITY as SHIPPING_CITY,
d.POSTAL_CODE as SHIPPING_ZIP_CODE,
d.REGION as SHIPPING_STATE,
d.COUNTRY as SHIPPING_COUNTRY
from
  (
    select distinct customer_id, shipping_address_id, billing_address_id from birdfactsdev.birdfacts_analytics.fact_sales
    where profit_center_country = 'United States'
    and sales_channel = 'Retail Brick & Mortar'
    and shipping_address_id not in
    (
      select distinct shipping_address_id
      from
      (
      select profit_center_name, shipping_address_id, count(distinct order_name) as num_orders
      from
      birdfactsdev.birdfacts_analytics.fact_sales
      where profit_center_country = 'United States'
      and sales_channel = 'Retail Brick & Mortar'
      group by 1, 2
      order by 3 desc
      )
      where num_orders > 10
      and shipping_address_id is not NULL
    )
  ) as a
  left join
  birdfactsdev.birdfacts_analytics.dim_customer as b
  on a.customer_id = b.id
  left join
  birdfactsdev.birdfacts_analytics.dim_address as c
  on a.billing_address_id = c.id
  left join
  birdfactsdev.birdfacts_analytics.dim_address as d
  on a.shipping_address_id = d.id
)
select row_number() over(partition by 1 order by 1)  as SEQUENCE_NUMBER,
* from emails_list_of_customers_with_shipping_address 
    where email not in (select distinct email from experian_data)
;"""

In [27]:
emails_list_of_customers_with_shipping_address = pd.DataFrame(cs.execute(emails_list_of_customers_with_shipping_address_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

In [30]:
print (emails_list_of_customers_with_shipping_address.shape)
emails_list_of_customers_with_shipping_address.head()

(2862, 16)


,SEQUENCE_NUMBER,EMAIL,FIRST_NAME,LAST_NAME,BILLING_ADDRESS_LINE_1,BILLING_ADDRESS_LINE_2,BILLING_CITY,BILLING_ZIP_CODE,BILLING_STATE,BILLING_COUNTRY,SHIPPING_ADDRESS_LINE_1,SHIPPING_ADDRESS_LINE_2,SHIPPING_CITY,SHIPPING_ZIP_CODE,SHIPPING_STATE,SHIPPING_COUNTRY
0,1,anamabas@outlook.com,Ana,Basurto,None,None,None,None,None,None,12116 Shoal Creek Drive,None,Frisco,75035,Texas,United States
1,2,sschutt13@me.com,Sarah,Schutt,None,None,None,None,None,None,2974 Roidt Drive,None,Stoughton,53589,Wisconsin,United States
2,3,shg1435@gmail.com,Steve,Goldman,None,None,None,None,None,None,155 W 66Th St,Apt 601,New York,10023,New York,United States
3,4,therese@timberlinemarketing-ss.com,Therese,Hurley,None,None,None,None,None,None,5837 Brooview Ave,None,Edina,55424,Minnesota,United States
4,5,dwilliams0543@gmail.com,Dana,Williams,None,None,None,None,None,None,20 Florida Rd,None,Towson,21204,Maryland,United States


In [28]:
# remove from emails list of customers without shipping address, the emails of those that we found in the ones with shipping address, so as to avoid deduplication
emails_list_of_customers_without_shipping_address = emails_list_of_customers_without_shipping_address[~emails_list_of_customers_without_shipping_address['EMAIL'].isin(emails_list_of_customers_with_shipping_address['EMAIL'].unique())]

In [113]:
print (emails_list_of_customers_without_shipping_address.shape)

(207058, 16)


In [142]:
# remove from simon list, the emails of those that we found in the ones with and without shipping address earlier, so as to avoid deduplication
simon_list_subscribed = simon_list_subscribed[~((simon_list_subscribed['email'].isin((emails_list_of_customers_with_shipping_address['EMAIL'].unique())))|(simon_list_subscribed['email'].isin((emails_list_of_customers_without_shipping_address['EMAIL'].unique()))))]

In [143]:
simon_list_subscribed.drop(['Unsubscribed?'], axis = 1, inplace = True)

In [144]:
simon_list_subscribed.rename({'email':'EMAIL'}, axis = 1, inplace = True)

In [145]:
simon_list_subscribed.reset_index(drop = True, inplace = True)

In [146]:
simon_list_subscribed.head()

,EMAIL
0,adammico@yahoo.com
1,cheriehornsby@mac.com
2,dhyoumans@gmail.com
3,heatherlorin@gmail.com
4,jean@parachutestrategy.co


In [147]:
for i in emails_list_of_customers_with_shipping_address.columns.tolist()[2:]:
    simon_list_subscribed[i] = np.nan

simon_list_subscribed['SEQUENCE_NUMBER'] = simon_list_subscribed.index + 1

In [149]:
simon_list_subscribed = simon_list_subscribed[emails_list_of_customers_with_shipping_address.columns.tolist()]

In [154]:
print(simon_list_subscribed.shape)
simon_list_subscribed.head()

(138936, 16)


,SEQUENCE_NUMBER,EMAIL,FIRST_NAME,LAST_NAME,BILLING_ADDRESS_LINE_1,BILLING_ADDRESS_LINE_2,BILLING_CITY,BILLING_ZIP_CODE,BILLING_STATE,BILLING_COUNTRY,SHIPPING_ADDRESS_LINE_1,SHIPPING_ADDRESS_LINE_2,SHIPPING_CITY,SHIPPING_ZIP_CODE,SHIPPING_STATE,SHIPPING_COUNTRY
0,1,adammico@yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,cheriehornsby@mac.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,dhyoumans@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,heatherlorin@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,jean@parachutestrategy.co,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
print(emails_list_of_customers_with_shipping_address.shape)
emails_list_of_customers_with_shipping_address.head()

(2862, 16)


,SEQUENCE_NUMBER,EMAIL,FIRST_NAME,LAST_NAME,BILLING_ADDRESS_LINE_1,BILLING_ADDRESS_LINE_2,BILLING_CITY,BILLING_ZIP_CODE,BILLING_STATE,BILLING_COUNTRY,SHIPPING_ADDRESS_LINE_1,SHIPPING_ADDRESS_LINE_2,SHIPPING_CITY,SHIPPING_ZIP_CODE,SHIPPING_STATE,SHIPPING_COUNTRY
0,1,anamabas@outlook.com,Ana,Basurto,None,None,None,None,None,None,12116 Shoal Creek Drive,None,Frisco,75035,Texas,United States
1,2,sschutt13@me.com,Sarah,Schutt,None,None,None,None,None,None,2974 Roidt Drive,None,Stoughton,53589,Wisconsin,United States
2,3,shg1435@gmail.com,Steve,Goldman,None,None,None,None,None,None,155 W 66Th St,Apt 601,New York,10023,New York,United States
3,4,therese@timberlinemarketing-ss.com,Therese,Hurley,None,None,None,None,None,None,5837 Brooview Ave,None,Edina,55424,Minnesota,United States
4,5,dwilliams0543@gmail.com,Dana,Williams,None,None,None,None,None,None,20 Florida Rd,None,Towson,21204,Maryland,United States


In [156]:
print(emails_list_of_customers_without_shipping_address.shape)
emails_list_of_customers_without_shipping_address.head()

(207058, 16)


,SEQUENCE_NUMBER,EMAIL,FIRST_NAME,LAST_NAME,BILLING_ADDRESS_LINE_1,BILLING_ADDRESS_LINE_2,BILLING_CITY,BILLING_ZIP_CODE,BILLING_STATE,BILLING_COUNTRY,SHIPPING_ADDRESS_LINE_1,SHIPPING_ADDRESS_LINE_2,SHIPPING_CITY,SHIPPING_ZIP_CODE,SHIPPING_STATE,SHIPPING_COUNTRY
0,1,fuliwang2003@yahoo.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,eric.shepherd1104@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,3,rfharty@comcast.net,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,4,timothy.s.keyes5@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,5,johnsonnli1111@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None


combine the three dataframes

In [157]:
emails_list_all = [emails_list_of_customers_without_shipping_address, emails_list_of_customers_with_shipping_address, simon_list_subscribed]

In [158]:
emails_list = pd.concat(emails_list_all, axis = 0, ignore_index = False)

load experian emails

In [162]:
experian_emails_query = """
select distinct email from birdfactsdev.birdfacts_analytics.experian_data
;"""

In [163]:
experian_emails = pd.DataFrame(cs.execute(experian_emails_query).fetchall(),
                           columns = ([col[0] for col in cs.description]))

remove experian emails from emails list

In [ ]:
emails_list = emails_list[~emails_list['EMAIL'].isin(experian_emails['EMAIL'].unique())]

remove emails that have 'allbirds' in them to avoid information on employees making purchases through allbirds account for purposes of work

In [173]:
emails_list = emails_list[~emails_list['EMAIL'].str.contains('allbirds', regex = False)]

sort and drop duplicates

In [180]:
emails_list = emails_list.sort_values(by = 'EMAIL')

In [184]:
emails_list.drop_duplicates(subset = 'EMAIL', keep = 'first', inplace = True)

resetting index and creating csv to upload to experian

In [186]:
emails_list.reset_index(drop = True, inplace = True)
emails_list['SEQUENCE_NUMBER'] = emails_list.index + 1

In [189]:
for col in emails_list.columns.tolist()[1:]:
    emails_list[col] = emails_list[col].str.replace('(','')
    emails_list[col] = emails_list[col].str.replace(')','')
    emails_list[col] = emails_list[col].str.replace('/','')
    emails_list[col] = emails_list[col].str.replace(',','')                                                                                                       

In [195]:
emails_list.fillna(np.nan, inplace = True)

In [209]:
emails_list.sort_values(by = 'EMAIL', inplace = True)
emails_list.drop_duplicates(subset = 'EMAIL', keep = 'first', inplace = True)

In [213]:
emails_list.to_csv("~/Documents/allbirds/experian_data/to_xpn/experian_off_sync_append_20200528.csv", index = False)

In [214]:
!wc -l ~/Documents/allbirds/experian_data/to_xpn/experian_off_sync_append_20200528.csv

  338839 /Users/vamsi/Documents/allbirds/experian_data/to_xpn/experian_off_sync_append_20200528.csv
